# GPU机器模型测试

- https://modelscope.cn/models/Qwen/Qwen2.5-7B-Instruct
  - context 128K 
  - 生成token 8K


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "/root/nas-private/huggingface_cache/Qwen25/Qwen2.5-7B-Instruct"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype="auto",
    device_map="auto"
)



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
# prepare the model input
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False 
    # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)


thinking content: 
content: Certainly! A Large Language Model (LLM) is a type of artificial intelligence system designed to understand and generate human-like text based on the input it receives. These models are typically deep learning models that have been trained on vast amounts of text data from the internet, books, and other sources. They can perform a wide range of natural language processing tasks, including but not limited to:

- **Text Generation**: Creating coherent paragraphs or even entire documents.
- **Translation**: Converting text from one language to another.
- **Summarization**: Condensing long texts into shorter summaries.
- **Question Answering**: Providing answers to questions based on given information.
- **Dialogue Generation**: Engaging in conversations with users.

LLMs are usually based on transformer architectures, which allow them to understand context and generate responses that are relevant and meaningful. The training process involves learning patterns an